In [2]:
import pandas as pd

# 데이터 로드
restaurant_data = pd.read_csv("C:/dacos_rec/crawling/df_meal (3).csv", encoding="euc-kr")
pub_data = pd.read_csv("C:/dacos_rec/crawling/df_drink(2).csv", encoding="euc-kr")
cafe_data = pd.read_csv("C:/dacos_rec/crawling/df_cafe(2).csv", encoding="euc-kr")

# 태그 옵션 정의
tag_options = {
    '식당': {
        '맛': ['음식이 맛있어요', '빵이 맛있어요'],
        '특별한 메뉴': ['특별한 메뉴가 있어요'],
        '메뉴 다양성': ['메뉴 구성이 알차요'],
        '신선도': ['재료가 신선해요', '잡내가 적어요', '고기 질이 좋아요'],
        '가성비': ['가성비가 좋아요', '양이 많아요'],
        '서비스': ['친절해요', '반찬이 잘 나와요', '음식이 빨리 나와요'],
        '청결': ['매장이 청결해요', '화장실이 깨끗해요'],
        '혼밥' : ['혼밥하기 좋아요']
    },
    '카페': {
        '커피': ['커피가 맛있어요'],
        '디저트': ['디저트가 맛있어요'],
        '청결': ['매장이 청결해요', '화장실이 깨끗해요'],
        '서비스': ['친절해요', '포장이 깔끔해요'],
        '분위기': ['아늑해요', '인테리어가 멋져요'],
        '공부하기 좋음음' : ['집중하기 좋아요', '좌석이 편해요', '매장이 넓어요']
    },
    '술집': {
        '맛': ['음식이 맛있어요'],
        '서비스': ['친절해요'],
        '다양한 술': ['술이 다양해요'],
        '분위기': ['음악이 좋아요'],
        '대화하기 좋음' : ['대화하기 좋아요']
    }
}

# 식당만 고정된 음식 종류
category_options = {
    '식당': ['한식', '중식', '일식', '양식', '육류', '아시안음식', '샐러드']
}

# 사용자 입력 처리 함수
def get_user_selection(prompt, choices):
    while True:
        print(prompt)
        for i, choice in enumerate(choices, 1):
            print(f"{i}. {choice}")
        try:
            selection = int(input("선택 번호를 입력하세요: ")) - 1
            if 0 <= selection < len(choices):
                return choices[selection]
            else:
                print(f"잘못된 입력입니다. 1부터 {len(choices)} 사이의 숫자를 입력하세요.")
        except ValueError:
            print("숫자를 입력하세요.")

# 사용자 입력
try:
    # 데이터 선택
    selected_category = get_user_selection(
        "데이터를 선택하세요 (1: 식당, 2: 술집, 3: 카페):", ["식당", "술집", "카페"]
    )

    if selected_category == "식당":
        selected_data = restaurant_data
        # 식당은 음식 종류 선택
        selected_food_category = get_user_selection(
            "음식 종류를 선택하세요:", category_options["식당"]
        )
        print(f"선택된 음식 종류: {selected_food_category}")
        # 필터링
        filtered_data = selected_data[selected_data['FoodCategory'] == selected_food_category]
    else:
        # 술집/카페는 음식 종류 없이 바로 진행
        if selected_category == "술집":
            selected_data = pub_data
        elif selected_category == "카페":
            selected_data = cafe_data
        filtered_data = selected_data  # 전체 데이터 사용
        selected_food_category = None  # 구분 없음

    # 태그 선택
    selected_tags = get_user_selection(
        "중요하게 생각하는 태그를 선택하세요:", list(tag_options[selected_category].keys())
    )
    print(f"선택된 태그: {selected_tags}")

except KeyError as e:
    print(f"선택지 처리 중 오류가 발생했습니다: {e}")
    raise

# 태그 기반 평균값 기준 필터링 함수
def rank_stores_by_threshold(data, selected_tags, tag_options):
    """
    선택한 태그를 만족하는 가게를 평균값 기준으로 필터링
    - data: 데이터프레임
    - selected_tags: 사용자가 선택한 태그 리스트
    - tag_options: 태그 매핑
    """
    # 선택한 태그의 실제 컬럼 이름 매핑
    valid_tags = []
    for tag in selected_tags:
        if tag in tag_options:
            valid_tags.extend(tag_options[tag])  # 선택한 태그의 모든 관련 컬럼 추가

    # 데이터프레임에 존재하는 태그만 필터링
    valid_tags = [t for t in valid_tags if t in data.columns]
    if not valid_tags:
        print(f"선택한 태그 {selected_tags}와 관련된 유효한 데이터가 없습니다.")
        return pd.DataFrame()

    # 유효한 태그들의 합계 계산
    data['Score'] = data[valid_tags].sum(axis=1)
    threshold = data['Score'].mean()  # 평균값 계산
    print(f"태그 리뷰 합계 평균값 기준 임계값: {threshold:.2f}")

    # 평균값 이상인 데이터 필터링
    filtered_data = data[data['Score'] >= threshold]
    return filtered_data

# 필터링된 결과 계산
ranked_results = rank_stores_by_threshold(filtered_data, selected_tags, tag_options[selected_category])

# 결과 출력
if not ranked_results.empty:
    print(f"평균값 기준 임계값 이상을 만족하는 가게:")
    print(ranked_results[['가게명', 'FoodCategory', 'Score']].head(10))  # 상위 10개 출력
else:
    print(f"선택한 태그 {selected_tags}에 대해 평균값 기준 이상 만족하는 가게가 없습니다.")



데이터를 선택하세요 (1: 식당, 2: 술집, 3: 카페):
1. 식당
2. 술집
3. 카페


중요하게 생각하는 태그를 선택하세요:
1. 커피
2. 디저트
3. 청결
4. 서비스
5. 분위기
6. 공부하기 좋음음
선택된 태그: 커피
선택한 태그 커피와 관련된 유효한 데이터가 없습니다.
선택한 태그 커피에 대해 평균값 기준 이상 만족하는 가게가 없습니다.
